# OFDM-GAN-SR Training on Google Colab

This notebook trains the CWGAN-GP model for OFDM signal reconstruction.

In [1]:
# Clone repository
!git clone https://github.com/orpheus016/ofdm-gan-sr.git
%cd ofdm-gan-sr

Cloning into 'ofdm-gan-sr'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 90 (delta 38), reused 78 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 590.91 KiB | 11.36 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/ofdm-gan-sr


In [2]:
# Install dependencies
!pip install -q -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.6/626.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 61.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==6.17.1, but you have ipykernel 7.1.0 which is incompatible.
jupyter-kernel-gateway 2.5.2 requires jupyter-client<8.0,>=5.2.0, but yo

In [ ]:
# Test model architectures before training
import torch
from models.generator import UNetGenerator
from models.discriminator import Discriminator

print("=" * 60)
print("MODEL VALIDATION TEST")
print("=" * 60)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Initialize models
print("\n[1/5] Initializing Generator...")
generator = UNetGenerator(
    input_channels=2,
    output_channels=2,
    base_channels=32,
    depth=5
).to(device)

print("[2/5] Initializing Discriminator...")
discriminator = Discriminator(
    input_channels=4,  # 2 (candidate) + 2 (condition)
    base_channels=32,
    num_layers=6
).to(device)

# Count parameters
gen_params = sum(p.numel() for p in generator.parameters())
disc_params = sum(p.numel() for p in discriminator.parameters())
print(f"\n✓ Generator parameters: {gen_params:,}")
print(f"✓ Discriminator parameters: {disc_params:,}")

# Test forward pass
print("\n[3/5] Testing Generator forward pass...")
batch_size = 4
noisy_signal = torch.randn(batch_size, 2, 1024).to(device)

try:
    fake_signal = generator(noisy_signal)
    assert fake_signal.shape == (batch_size, 2, 1024), \
        f"Generator output shape mismatch: {fake_signal.shape}"
    print(f"✓ Generator output shape: {tuple(fake_signal.shape)}")
except Exception as e:
    print(f"❌ Generator test FAILED: {e}")
    raise

# Test discriminator
print("\n[4/5] Testing Discriminator forward pass...")
clean_signal = torch.randn(batch_size, 2, 1024).to(device)
condition = noisy_signal  # Use noisy as condition

try:
    # Concatenate along channel dimension
    disc_input = torch.cat([fake_signal.detach(), condition], dim=1)
    score = discriminator(disc_input)
    print(f"✓ Discriminator output shape: {tuple(score.shape)}")
    print(f"✓ Score range: [{score.min().item():.3f}, {score.max().item():.3f}]")
except Exception as e:
    print(f"❌ Discriminator test FAILED: {e}")
    raise

# Memory usage
print("\n[5/5] Checking memory usage...")
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(device) / 1e9
    reserved = torch.cuda.memory_reserved(device) / 1e9
    print(f"✓ GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")

print("\n" + "=" * 60)
print("✅ ALL TESTS PASSED - Models are ready for training!")
print("=" * 60)

MODEL VALIDATION TEST

✓ Device: cpu

[1/5] Initializing Generator...


TypeError: UNetGenerator.__init__() got an unexpected keyword argument 'in_channels'

In [ ]:
# Start training with fixed configuration
!python train.py --synthetic --epochs 100 --lr 0.0001 --batch_size 16

In [ ]:
# Optional: Monitor training with TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs/

In [ ]:
# Optional: Save checkpoints to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create backup directory
!mkdir -p /content/drive/MyDrive/ofdm-gan-checkpoints

# Copy checkpoints (run this after training completes or periodically)
!cp -r checkpoints/* /content/drive/MyDrive/ofdm-gan-checkpoints/
!cp -r runs/* /content/drive/MyDrive/ofdm-gan-checkpoints/runs/

print("✓ Checkpoints saved to Google Drive")

In [6]:
!python train.py --synthetic --epochs 100 --lr 0.0001

2025-12-17 08:12:33.209592: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 08:12:33.217639: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 08:12:33.241777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765959153.292940   13039 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765959153.306888   13039 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765959153.344960   13039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin